<a href="https://colab.research.google.com/github/Hind117/DefenseMechanisms_Project/blob/main/DefenseMechanisms_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
import gensim
import csv
nltk.download('punkt')
#import fasttext

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("/content/XSS_dataset.csv")

df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [3]:
df.shape

(13686, 3)

In [4]:
df['Label'].value_counts(normalize = True)

1    0.538726
0    0.461274
Name: Label, dtype: float64

In [5]:
df.duplicated(['Sentence']).sum()

2769

In [6]:
df.columns

Index(['Unnamed: 0', 'Sentence', 'Label'], dtype='object')

In [7]:
df.pop('Unnamed: 0')

0            0
1            1
2            2
3            3
4            4
         ...  
13681    13681
13682    13682
13683    13683
13684    13684
13685    13685
Name: Unnamed: 0, Length: 13686, dtype: int64

In [8]:
df.columns

Index(['Sentence', 'Label'], dtype='object')

In [9]:
df.duplicated().sum()

2769

In [10]:
df.drop_duplicates(inplace = True)

In [11]:
df.shape

(10917, 2)

In [12]:
df['Label'].value_counts(normalize = True)

1    0.670789
0    0.329211
Name: Label, dtype: float64

In [13]:
def tokenize(payload):
    
    tokens=payload.lower()

    
    Dict = { '<': 'less', '>': 'gret', '/': 'slsh', '=': 'eql', '"': 'dqout',
             ':': 'coln', '(': 'opbrk', ')': 'cbrk', '#': 'hash', '&': 'and',
             '-': 'hyph', '_': 'udsc', '{': 'opcrl', '}': 'clcrl', '@': 'at',
             '^': 'pow', '.': 'dot', '%': 'perc', '$': 'dolr', '!': 'exlm', 
             '*': 'all', ';': 'semicoln', '\\': 'bslsh', '`': 'backtik', ',': 'coma',
             '‘': 'opapost', '’': 'capost', '+': 'plus', '~': 'tlde', '?': 'qusm',
             '[': 'opbrnt', ']': 'cbrnt', '\'': 'sqout', '|': 'pipe', '–': 'dhyph' }
    
       
    for token in tokens:
        if token in Dict:
            Dict[token] = ' '+Dict[token]+' '
            tokens=tokens.replace(token, Dict[token])
        elif re.search("^([0-9]+)$", token):
            tokens=tokens.replace(token, ' int ')
           
    tokens=word_tokenize(tokens)
    return " ".join(tokens)

        
text = '<svg><meta onload=alert(1)></meta>'
tokenize(text)

'less svg gret less meta onload eql alert opbrk int cbrk gret less slsh meta gret'

In [14]:
df['tokenized_payload'] = df['Sentence'].apply(tokenize)

In [15]:
df.head()

,Sentence,Label,tokenized_payload
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0,less li gret less a href eql dqout slsh wiki s...
1,"<tt onmouseover=""alert(1)"">test</tt>",1,less tt onmouseover eql dqout alert opbrk int ...
2,"\t </span> <span class=""reference-text"">Steeri...",0,less slsh span gret less span class eql dqout ...
3,"\t </span> <span class=""reference-text""><cite ...",0,less slsh span gret less span class eql dqout ...
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0,less slsh span gret dot less a href eql dqout ...


In [16]:
tokenized_payloads = df['tokenized_payload']

In [17]:
tokenized_payloads

0        less li gret less a href eql dqout slsh wiki s...
1        less tt onmouseover eql dqout alert opbrk int ...
2        less slsh span gret less span class eql dqout ...
3        less slsh span gret less span class eql dqout ...
4        less slsh span gret dot less a href eql dqout ...
                               ...                        
13678    less li gret less a href eql dqout slsh wiki s...
13681    less img onpointerenter eql alert opbrk int cb...
13682    less source onbeforepaste eql dqout alert opbr...
13683    less div draggable eql dqout true dqout conten...
13684    less li gret less cite id eql dqout citerefdom...
Name: tokenized_payload, Length: 10917, dtype: object

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

X = df['tokenized_payload']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42, shuffle=True, stratify=y)

model = LogisticRegression()

pipeline_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', model)])

pipeline_model.fit(X_train, y_train)

print('Accuracy:', pipeline_model.score(X_test, y_test)*100)

y_pred = pipeline_model.predict(X_test)

print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(conf_matrix)

Accuracy: 99.77106227106228
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       719
           1       1.00      1.00      1.00      1465

    accuracy                           1.00      2184
   macro avg       1.00      1.00      1.00      2184
weighted avg       1.00      1.00      1.00      2184

[[ 717    2]
 [   3 1462]]


In [19]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes_model = MultinomialNB()

pipeline_naive_bayes_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', naive_bayes_model)])

pipeline_naive_bayes_model.fit(X_train, y_train)

print('Accuracy:', pipeline_naive_bayes_model.score(X_test, y_test)*100)

y_pred_naive_bayes_model = pipeline_naive_bayes_model.predict(X_test)

print(classification_report(y_test, y_pred_naive_bayes_model))

nb_conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_pred_naive_bayes_model)
print(nb_conf_matrix)



Accuracy: 98.3974358974359
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       719
           1       0.98      1.00      0.99      1465

    accuracy                           0.98      2184
   macro avg       0.99      0.98      0.98      2184
weighted avg       0.98      0.98      0.98      2184

[[ 689   30]
 [   5 1460]]


In [20]:
from sklearn.svm import SVC

SVC_model = SVC()

pipeline_SVC_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', SVC_model)])

pipeline_SVC_model.fit(X_train, y_train)

print('Accuracy:', pipeline_SVC_model.score(X_test, y_test)*100)

y_SVC_pred = pipeline_SVC_model.predict(X_test)

print(classification_report(y_test, y_SVC_pred))

svc_conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_SVC_pred)
print(svc_conf_matrix)

Accuracy: 99.86263736263736
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       719
           1       1.00      1.00      1.00      1465

    accuracy                           1.00      2184
   macro avg       1.00      1.00      1.00      2184
weighted avg       1.00      1.00      1.00      2184

[[ 718    1]
 [   2 1463]]


In [21]:
from sklearn.ensemble import RandomForestClassifier

RF_model = RandomForestClassifier()

pipeline_RF_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', RF_model)])

pipeline_RF_model.fit(X_train, y_train)

print('Accuracy:', pipeline_RF_model.score(X_test, y_test)*100)

y_RF_pred = pipeline_RF_model.predict(X_test)

print(classification_report(y_test, y_RF_pred))

rf_conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_RF_pred)
print(rf_conf_matrix)

Accuracy: 99.72527472527473
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       719
           1       1.00      1.00      1.00      1465

    accuracy                           1.00      2184
   macro avg       1.00      1.00      1.00      2184
weighted avg       1.00      1.00      1.00      2184

[[ 718    1]
 [   5 1460]]


In [22]:
from sklearn.tree import DecisionTreeClassifier

DT_model = DecisionTreeClassifier()


#word embedding
pipeline_DT_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', DT_model)])

pipeline_DT_model.fit(X_train, y_train) #train the model

#testing and calculating the accuracy
print('Accuracy:', pipeline_DT_model.score(X_test, y_test)*100)

#predicting
y_DT_pred = pipeline_DT_model.predict(X_test)

print(classification_report(y_test, y_DT_pred))

dt_conf_matrix = confusion_matrix(y_true=y_test, y_pred=y_DT_pred)
print(dt_conf_matrix)

Accuracy: 99.54212454212454
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       719
           1       1.00      1.00      1.00      1465

    accuracy                           1.00      2184
   macro avg       0.99      1.00      0.99      2184
weighted avg       1.00      1.00      1.00      2184

[[ 716    3]
 [   7 1458]]


FastText Model

In [23]:
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True, stratify=y)
train.to_csv('/content/train.csv',index=False)
test.to_csv('/content/test.csv',index=False)

In [24]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [25]:
train.head()

,Sentence,Label,tokenized_payload
0,"<li class=""toctree-l1""><a class=""reference int...",0,less li class eql dqout toctree hyph l int dqo...
1,"<div draggable=""true"" contenteditable>drag me<...",1,less div draggable eql dqout true dqout conten...
2,"<button draggable=""true"" ondragleave=""alert(1)...",1,less button draggable eql dqout true dqout ond...
3,<map onpointerout=alert(1)>XSS</map>,1,less map onpointerout eql alert opbrk int cbrk...
4,"<!DOCTYPE html><html class=""no-js"" lang=""en"" >",0,less exlm doctype html gret less html class eq...


In [26]:
train.pop('Sentence')

0       <li class="toctree-l1"><a class="reference int...
1       <div draggable="true" contenteditable>drag me<...
2       <button draggable="true" ondragleave="alert(1)...
3                    <map onpointerout=alert(1)>XSS</map>
4         <!DOCTYPE html><html class="no-js" lang="en" > 
                              ...                        
8728    <style>:target {transform: rotate(180deg);}</s...
8729    <menuitem id=x tabindex=1 onfocus=alert(1)></m...
8730    <li><cite id="CITEREFMaker2006" class="citatio...
8731                   <s onmousemove="alert(1)">test</s>
8732    <font id=x tabindex=1 onactivate=alert(1)></font>
Name: Sentence, Length: 8733, dtype: object

In [27]:
train.head()

,Label,tokenized_payload
0,0,less li class eql dqout toctree hyph l int dqo...
1,1,less div draggable eql dqout true dqout conten...
2,1,less button draggable eql dqout true dqout ond...
3,1,less map onpointerout eql alert opbrk int cbrk...
4,0,less exlm doctype html gret less html class eq...


In [28]:
#
train['Label'] = train['Label'].map({0:'__label__Nonmalicious', 1:'__label__Malicious'})

In [29]:
train.head()

,Label,tokenized_payload
0,__label__Nonmalicious,less li class eql dqout toctree hyph l int dqo...
1,__label__Malicious,less div draggable eql dqout true dqout conten...
2,__label__Malicious,less button draggable eql dqout true dqout ond...
3,__label__Malicious,less map onpointerout eql alert opbrk int cbrk...
4,__label__Nonmalicious,less exlm doctype html gret less html class eq...


In [30]:
test['Label'] = test['Label'].map({0:'__label__Nonmalicious', 1:'__label__Malicious'})

In [31]:
test.head()

,Sentence,Label,tokenized_payload
0,<keygen onpointerdown=alert(1)>XSS</keygen>,__label__Malicious,less keygen onpointerdown eql alert opbrk int ...
1,<link id=x tabindex=1 ondeactivate=alert(1)></...,__label__Malicious,less link id eql x tabindex eql int ondeactiva...
2,"<li id=""cite_note-sak2014-269""><span class=""mw...",__label__Nonmalicious,less li id eql dqout cite udsc note hyph sak i...
3,"<pre oncontextmenu=""alert(1)"">test</pre>",__label__Malicious,less pre oncontextmenu eql dqout alert opbrk i...
4,"<center onkeydown=""alert(1)"" contenteditable>t...",__label__Malicious,less center onkeydown eql dqout alert opbrk in...


In [32]:
test.pop('Sentence')

0             <keygen onpointerdown=alert(1)>XSS</keygen>
1       <link id=x tabindex=1 ondeactivate=alert(1)></...
2       <li id="cite_note-sak2014-269"><span class="mw...
3                <pre oncontextmenu="alert(1)">test</pre>
4       <center onkeydown="alert(1)" contenteditable>t...
                              ...                        
2179    <ruby onkeypress="alert(1)" contenteditable>te...
2180                        <input id=x onfocus=alert(1)>
2181    <ul><li><a href="#CITEREFLucas1961">Lucas 1961...
2182               <small onclick="alert(1)">test</small>
2183    <section id=x tabindex=1 ondeactivate=alert(1)...
Name: Sentence, Length: 2184, dtype: object

In [33]:
test.head()

,Label,tokenized_payload
0,__label__Malicious,less keygen onpointerdown eql alert opbrk int ...
1,__label__Malicious,less link id eql x tabindex eql int ondeactiva...
2,__label__Nonmalicious,less li id eql dqout cite udsc note hyph sak i...
3,__label__Malicious,less pre oncontextmenu eql dqout alert opbrk i...
4,__label__Malicious,less center onkeydown eql dqout alert opbrk in...


In [34]:
#convert CSV files to text files
train[['Label', 'tokenized_payload']].to_csv('/content/train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")
test[['Label', 'tokenized_payload']].to_csv('/content/test.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")



In [35]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import fasttext

In [37]:
#train fasttext model
fasttext_model = fasttext.train_supervised('/content/train.txt')

In [51]:


Number_of_Samples, Precision, Recall = fasttext_model.test('/content/test.txt') #test the model

F_Measure = ((2 * Precision * Recall) / (Precision + Recall))
print(F_Measure*100)



(2184, 0.9981684981684982, 0.9981684981684982)

In [49]:
#calculate the accuracy
df["predicted"] = df["tokenized_payload"].apply(lambda x: fasttext_model.predict(x)[0][0])
df['predicted'] = df['predicted'].map({'__label__Nonmalicious':0, '__label__Malicious':1})


[[TP, FP], [FN, TN]] = confusion_matrix(df["Label"], df["predicted"])

Accuracy = (TP+TN)/(TP+TN+FP+FN)
print(Accuracy)


0.9979847943574242


In [40]:
print(tokenize('<svg><a><animate attributeName=href values=javascript:alert(1) /><text x=20 y=20>Click me</text></a></svg>'))


less svg gret less a gret less animate attributename eql href values eql javascript coln alert opbrk int cbrk slsh gret less text x eql int int y eql int int gret click me less slsh text gret less slsh a gret less slsh svg gret


In [41]:
fasttext_model.predict('less svg gret less a gret less animate attributename eql href values eql javascript coln alert opbrk int cbrk slsh gret less text x eql int int y eql int int gret click me less slsh text gret less slsh a gret less slsh svg gret')

(('__label__Malicious',), array([0.97377121]))

In [42]:
print(tokenize('<x ondrag=alert(1)>drag this!'))

less x ondrag eql alert opbrk int cbrk gret drag this exlm


In [43]:
fasttext_model.predict('less x ondrag eql alert opbrk int cbrk gret drag this exlm')

(('__label__Malicious',), array([1.00000811]))